# Trigger files
This notebook is intended to create the descriptive, streams information and timestamps as '.csv' files, and trigger files.
- Descriptive files contain information about the recordings such as duration, sampling rate, names of streams collected, type, etc.
- Stream information files contain all data gathered during recording with respective timestamps (e.g., head and eye tracking, object names)
- The trigger files contain information about the time when an image was shown, the tpe  of  ima required for the eeg data analysis.

In [169]:
import copy
import datetime
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from matplotlib.ticker import FormatStrFormatter

import pandas as pd
import pyxdf
import seaborn as sns
from scipy.signal import find_peaks
from operator import itemgetter
from tqdm.notebook import tqdm

In [170]:
# path to data stored
r_path = "data"
# path to store trigger data
t_path = r_path + '/triggers'
# path to store eye-tracking data
e_path = r_path + '/eye_tracking'

files = os.listdir(r_path)  # get all files from the folder "data"
files.sort()  # sort them alphabetically
recordings = {}
file_names = []
for i, file in enumerate(files):  # store and display all files
    if file.endswith('.xdf'):
        file_names.append(file)
        created = os.path.getmtime(f"data/{file}")  # creation timestamp
        created = datetime.datetime.fromtimestamp(created)  # translate as datetime
        created = created.strftime("%d.%m.%Y %H:%M")  # arrange it
        recordings[i] = {"file": file, "created": created}
# print(file_names)
files = [f.split(".")[0] for f in files]
print("Included:")
display(recordings)

Included:


{1: {'file': '01_room1_pilot1.xdf', 'created': '14.10.2022 21:32'},
 2: {'file': '39_room1_251022.xdf', 'created': '25.10.2022 16:40'},
 3: {'file': '40_room1_261022.xdf', 'created': '26.10.2022 16:54'}}

## 1. Load data

In [179]:
# check streams for recording 0
file_to_use = file_names[1]
streams, _ = pyxdf.load_xdf(f"data/{file_to_use}")

In [180]:
# stream channel names in recording 0
s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
s_channels

{'ImagesOrder': 0,
 'ValidationError': 1,
 'HeadTracking': 2,
 'openvibeSignal': 3,
 'EyeTrackingLocal': 4,
 'EyeTrackingWorld': 5,
 'HitObjectPositions': 6,
 'ImageInfo': 7,
 'Visual': 8,
 'HitObjectNames': 9,
 'HitPositionOnObjects': 10}

In [147]:
streams[4]['footer']['info']['first_timestamp']

['181523.6270941']

In [188]:
for i in streams:
    print(i['info']['source_id'])


['fb70e791-699d-4b1a-8a3f-59e6c10c736b']
['fb70e791-699d-4b1a-8a3f-59e6c10c736b']
['fb70e791-699d-4b1a-8a3f-59e6c10c736b']
['fb70e791-699d-4b1a-8a3f-59e6c10c736b']
['(0x7bc2b1a8, 0xf2c8e25b)']
['fb70e791-699d-4b1a-8a3f-59e6c10c736b']
['fb70e791-699d-4b1a-8a3f-59e6c10c736b']
['(0x9bb8cbee, 0xc755da37)']
['fb70e791-699d-4b1a-8a3f-59e6c10c736b']
['fb70e791-699d-4b1a-8a3f-59e6c10c736b']
['fb70e791-699d-4b1a-8a3f-59e6c10c736b']
['fb70e791-699d-4b1a-8a3f-59e6c10c736b']


In [112]:
len(streams[2]['time_stamps'])

111180

In [237]:
def describe_recordings(streams, file_name=''):
    ch_keys = ["name","type", "channel_count","channel_format","nominal_srate","desc","effective_srate","hostname", "created_at"]
    s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}

    for i, ch_name in enumerate(s_channels):
        if ch_name in 'openvibeSignal':
            # save uid and append to csv
            uid = streams[i]['info']['uid'][0]
            # calculate the recording duration based on first and last time_stamps
            duration = (streams[i]['time_stamps'][-1]- streams[i]['time_stamps'][0])/60
        if ch_name in 'Visual':
            # calculate the duration for the 1st unity half block
            block_1_duration  = (streams[i]['time_stamps'][30000] - streams[i]['time_stamps'][0])/60
            # Unity sampling rate (fps) for 1rst half block
            fps = 1 / (block_1_duration * 60 / 30000)

    df = pd.DataFrame(columns=np.concatenate([['uid','file_name','duration (min)','unity_fps','stream_id'], ch_keys]))
    id_o = []
    for id, stream in enumerate(streams):
        id_o.append(id)
        items = itemgetter(*ch_keys)(dict(stream['info']))
        items = [(', '.join(list(item[0].keys())) if 'desc' in ch_keys[i] else (
                  item[0] if isinstance(item, list) else item)) for i, item in enumerate(items)]
        df = df.append(pd.Series(items, index=ch_keys), ignore_index=True)

    df['uid'] = np.resize(uid,len(streams))
    df['file_name'] = np.resize(file_name,len(streams))
    df['duration (min)'] = np.resize(duration,len(streams))
    df['unity_fps'] = np.resize(fps,len(streams))
    df['stream_id'] = id_o

    return df

In [238]:
desc_streams = describe_recordings(streams, file_name=file_to_use)
desc_streams

,uid,file_name,duration (min),unity_fps,stream_id,name,type,channel_count,channel_format,nominal_srate,desc,effective_srate,hostname,created_at
0,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,0,HitObjectNames,Markers,1,string,0.000000000000000,HON,0,ml03,634.4911906000000
1,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,1,ImagesOrder,Markers,2,string,0.000000000000000,"arrayOfImages, orderOfImageIndexesToDisplay",0,ml03,634.4975368000000
2,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,2,Visual,Markers,3,float32,0.000000000000000,"cFrame, displayStatus, worldTime",0,ml03,634.4950234000000
3,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,3,ImageInfo,Markers,2,string,0.000000000000000,"blockNumber, imageName",0,ml03,634.4963238000000
4,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,4,openvibeSignal,signal,72,float32,1024.000000000000,channels,1023.999998,wd-left,104.2210639000000
5,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,5,HeadTracking,Markers,6,float32,0.000000000000000,"HToriginX, HToriginY, HToriginZ, HTdirectionX,...",0,ml03,634.4899724000001
6,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,6,EyeTrackingWorld,Markers,10,float32,0.000000000000000,"ETWTime, ETWoriginX, ETWoriginY, ETWoriginZ, E...",0,ml03,634.4875234000000
7,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,7,openvibeMarkers,Markers,1,int32,0.000000000000000,channels,0,wd-left,104.2298864000000
8,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,8,EyeTrackingLocal,Markers,6,float32,0.000000000000000,"ETLoriginX, ETLoriginY, ETLoriginZ, ETLdirecti...",0,ml03,634.4887757000000
9,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,39_room1_251022.xdf,22.374463,89.998318,9,HitPositionOnObjects,Markers,3,float32,0.000000000000000,"HPOOX, HPOOY, HPOOZ",0,ml03,634.4937609999999


In [90]:
# 3730.732377192173
# 3730.73246817
ref_ts = (streams[4]['time_stamps'][-1] - streams[4]['time_stamps'][0]) / 60
ref_ts
# eeg_ts1 = float(ref_ts) -1
# eeg_ts1

22.374462932040434

## 2. Create dataframe from streams

In [221]:
def get_streams_data(streams, streams_keep=['ImageInfo','Visual'], time_ref_stream='openvibeSignal'):
    """
    :param streams: streams after loading from .xdf file
    :param streams_keep: str. of the stream names to keep
    :param time_ref_stream: str. name of the eeg  signal used used for time correction and uid creation
    :return: df containing the time_stamps and stream_data as columns for each stream to keep
    """
    s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
    eeg_u = s_channels[time_ref_stream]
    eeg_first_ts = float(streams[eeg_u]['time_stamps'][0])
    eeg_all_ts = streams[eeg_u]['time_stamps'] # all eeg timestamps
    uid = streams[eeg_u]['info']['uid'][0]

    data = pd.DataFrame()
    for i, ch_name in enumerate(streams_keep):
        # get all current streams with their positions on the recording
        # example: {'ImagesOrder': 0, 'ValidationError': 1, 'HeadTracking': 2}
        s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
        u = s_channels[ch_name]
        # save the recording UID and append to df
        data['uid'] = np.resize(uid,len(data))

        # check the type and length of data arrays and get only 1 value of the array
        stream_data = streams[u]['time_series']
        # double check keys on each stream to make sure they are all appended to df
        # print(f"Stream {ch_name} keys: {streams[u]['info']['desc'][0].keys()}")
        # check stram_data is of kind np.array()
        if isinstance(stream_data, (list,pd.core.series.Series,np.ndarray)):
            # access all stream names in dictionary's 'info' description
            for i, key in enumerate(streams[u]['info']['desc'][0].keys()):
                # save each dict key as column to df
                stream_data = pd.DataFrame(streams[u]['time_series'])[i]
                data[f"{key}_{ch_name}"] = stream_data


        # get timestamps and attach them as column to df
        time_stamps =  streams[u]['time_stamps']
        data = pd.concat([data, pd.DataFrame(time_stamps, columns=[f"time_stamps_{ch_name}"])], axis=1)
        # save timestamps after normalising them to zero
        data[f"corrected_tstamps_{ch_name}"] = data[f"time_stamps_{ch_name}"] - eeg_first_ts

    return data

In [222]:
df = get_streams_data(streams, streams_keep=['ImageInfo','Visual'], time_ref_stream='openvibeSignal')
df

,uid,blockNumber_ImageInfo,imageName_ImageInfo,time_stamps_ImageInfo,corrected_tstamps_ImageInfo,cFrame_Visual,displayStatus_Visual,worldTime_Visual,time_stamps_Visual,corrected_tstamps_Visual
0,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3730.998672,0.266204,3841.0,2.0,91.064651,3730.998659,0.266191
1,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3731.009716,0.277248,3842.0,2.0,91.075706,3731.009707,0.277238
2,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3731.020642,0.288174,3843.0,2.0,91.086632,3731.020630,0.288162
3,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3731.031608,0.299140,3844.0,2.0,91.097603,3731.031599,0.299130
4,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3731.042741,0.310273,3845.0,2.0,91.108734,3731.042731,0.310263
...,...,...,...,...,...,...,...,...,...,...
111175,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,1,img.1600x1000.date.2022-10-07_17-31-45.hitname...,5066.135132,1335.402663,114850.0,0.0,1426.197998,5066.135121,1335.402653
111176,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,1,img.1600x1000.date.2022-10-07_17-31-45.hitname...,5066.146061,1335.413592,114851.0,0.0,1426.208862,5066.146050,1335.413581
111177,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,1,img.1600x1000.date.2022-10-07_17-31-45.hitname...,5066.157166,1335.424698,114852.0,0.0,1426.219971,5066.157156,1335.424688
111178,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,1,img.1600x1000.date.2022-10-07_17-31-45.hitname...,5066.168250,1335.435782,114853.0,0.0,1426.231079,5066.168240,1335.435772


In [219]:
# quick statistics to know if all rows have same length
df.describe()
# df['HON_HitObjectNames'].isnull().sum()
# df['HON_HitObjectNames'].unique()

,time_stamps_ImageInfo,corrected_tstamps_ImageInfo,cFrame_Visual,displayStatus_Visual,worldTime_Visual,time_stamps_Visual,corrected_tstamps_Visual
count,111180.000000,111180.000000,111180.000000,111180.000000,111180.000000,111180.000000,111180.000000
mean,4393.360486,662.628018,59361.429688,1.440394,753.423584,4393.360477,662.628008
std,394.274395,394.274395,32028.070312,1.333862,394.272064,394.274395,394.274395
min,3730.998672,0.266204,3841.000000,0.000000,91.064651,3730.998659,0.266191
25%,4039.835421,309.102953,31648.750000,0.000000,399.900681,4039.835412,309.102943
50%,4381.357276,650.624808,59382.500000,1.000000,741.421722,4381.357267,650.624799
75%,4757.342356,1026.609887,87046.250000,2.000000,1117.405884,4757.342347,1026.609879
max,5066.179362,1335.446894,114854.000000,5.000000,1426.242188,5066.179353,1335.446884


In [223]:
# inspect the eye_tracking data
df_eye = get_streams_data(streams, streams_keep=['HitObjectNames', 'HitPositionOnObjects','HeadTracking','EyeTrackingWorld','EyeTrackingLocal'])
df_eye

,uid,HON_HitObjectNames,time_stamps_HitObjectNames,corrected_tstamps_HitObjectNames,HPOOX_HitPositionOnObjects,HPOOY_HitPositionOnObjects,HPOOZ_HitPositionOnObjects,time_stamps_HitPositionOnObjects,corrected_tstamps_HitPositionOnObjects,HToriginX_HeadTracking,...,time_stamps_EyeTrackingWorld,corrected_tstamps_EyeTrackingWorld,ETLoriginX_EyeTrackingLocal,ETLoriginY_EyeTrackingLocal,ETLoriginZ_EyeTrackingLocal,ETLdirectionX_EyeTrackingLocal,ETLdirectionY_EyeTrackingLocal,ETLdirectionZ_EyeTrackingLocal,time_stamps_EyeTrackingLocal,corrected_tstamps_EyeTrackingLocal
0,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,FixationCollider,3730.997349,0.264881,0.025130,1.446558,9.418699,3730.997353,0.264885,0.017377,...,3730.997272,0.264803,0.031408,0.001798,-0.025572,0.002686,-0.093506,0.995605,3730.998010,0.265542
1,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,FixationCollider,3731.008460,0.275992,0.082237,1.446519,9.423999,3731.008465,0.275996,0.017391,...,3731.008376,0.275908,0.031048,0.001815,-0.025599,0.008621,-0.093369,0.995590,3731.009143,0.276674
2,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,FixationCollider,3731.019443,0.286975,0.045631,1.432317,9.416549,3731.019446,0.286978,0.017432,...,3731.019357,0.286889,0.026963,0.001906,-0.025647,0.005142,-0.094482,0.995499,3731.020026,0.287558
3,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,FixationCollider,3731.030531,0.298062,0.034770,1.427868,9.414793,3731.030533,0.298065,0.017440,...,3731.030461,0.297993,0.023141,0.001992,-0.025693,0.004288,-0.094803,0.995483,3731.031090,0.298622
4,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,FixationCollider,3731.041669,0.309201,0.010570,1.432112,9.414815,3731.041672,0.309203,0.017486,...,3731.041608,0.309140,0.016942,0.002129,-0.025755,0.002258,-0.094482,0.995514,3731.042199,0.309731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100492,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,FixationCollider,5066.133986,1335.401517,-0.173868,1.057811,9.479301,5066.133989,1335.401521,0.029180,...,5066.133909,1335.401441,-0.000950,0.003270,-0.027301,0.137268,-0.136169,0.981125,5066.134563,1335.402095
100493,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,FixationCollider,5066.144915,1335.412447,-0.170069,1.069861,9.472626,5066.144917,1335.412449,0.029200,...,5066.144838,1335.412370,-0.000823,0.003266,-0.027293,0.137665,-0.135559,0.981155,5066.145489,1335.413020
100494,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,FixationCollider,5066.156074,1335.423606,-0.169865,1.070280,9.472397,5066.156075,1335.423607,0.029205,...,5066.156014,1335.423545,-0.000680,0.003261,-0.027286,0.137695,-0.135757,0.981110,5066.156599,1335.424131
100495,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,FixationCollider,5066.167128,1335.434660,-0.171499,1.069171,9.473394,5066.167129,1335.434661,0.029212,...,5066.167067,1335.434599,-0.000224,0.003246,-0.027267,0.137497,-0.136063,0.981110,5066.167650,1335.435181


In [151]:
# df_eye.describe()
df_eye.isnull().sum()

UID                                 0
HON_HitObjectNames                  0
time_stamps_HitObjectNames          0
HPOOX_HitPositionOnObjects          0
HPOOY_HitPositionOnObjects          0
HPOOZ_HitPositionOnObjects          0
time_stamps_HitPositionOnObjects    0
HToriginX_HeadTracking              0
HToriginY_HeadTracking              0
HToriginZ_HeadTracking              0
HTdirectionX_HeadTracking           0
HTdirectionY_HeadTracking           0
HTdirectionZ_HeadTracking           0
time_stamps_HeadTracking            0
ETWTime_EyeTrackingWorld            0
ETWoriginX_EyeTrackingWorld         0
ETWoriginY_EyeTrackingWorld         0
ETWoriginZ_EyeTrackingWorld         0
ETWdirectionX_EyeTrackingWorld      0
ETWdirectionY_EyeTrackingWorld      0
ETWdirectionZ_EyeTrackingWorld      0
leftBlink_EyeTrackingWorld          0
rightBlink_EyeTrackingWorld         0
valid_EyeTrackingWorld              0
time_stamps_EyeTrackingWorld        0
dtype: int64

## 3. Create triggers
- For each initial time an image was shown, we want to keep the type of object it was (i.e., face, object, body) as a separate column.
- Additional triggers contain the rotation and distance the specific object was with respect to the player at the time the free-viewing walk took place.
- __Note:__ We want the triggers only once to denote the initial time the image was shown.

In [224]:
def create_triggers(df):
    # save the names of the object, body or face shown in the image
    df['ob_names'] =  df.apply(lambda x: x["imageName_ImageInfo"].split(".")[5] if len(x["imageName_ImageInfo"].split(".")) > 7 else '', axis=1)
    # create the triggers to mark the start of image displaying
    # check when there is a change from image, canvas, fixationCross
    df['shift'] = df['imageName_ImageInfo'].shift(1) != df['imageName_ImageInfo']
    # save the starting time (aka 'latency') when image is displayed
    df['latency'] = df.apply(lambda x: x['corrected_tstamps_Visual'] if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    # save the type of image displaying (face, object, body)
    df['type'] = df.apply(lambda x: 'face' if x['shift'] and 'face' in x['imageName_ImageInfo'].lower()
                                            else ('body' if x['shift'] and 'npc' in x['imageName_ImageInfo'].lower()
                                            else ('object' if x['shift'] and 'rotation' in x['imageName_ImageInfo'].lower()
                                            and 'face|npc' not in x['imageName_ImageInfo'].lower()
                                            else '')), axis=1)
    # define the triggers for rotation, distance, and block
    df['rotation'] = df.apply(lambda x: 'r_' + x["imageName_ImageInfo"].split(".")[7] if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df['distance'] = df.apply(lambda x: 'd_' + x["imageName_ImageInfo"].split(".")[9] if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df['block'] = df.apply(lambda x: 'b_' + str(x["blockNumber_ImageInfo"]) if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    # select the trigger columns and non empty rows
    df_sel = df[['latency','type','rotation','distance','block']]
    df_triggers = df_sel[df_sel['latency'] != '']
    # save first uid for later usage
    uid = df['uid'][0]
    return df, df_triggers, uid

In [225]:
df, df_triggers, uid = create_triggers(df)
df

,uid,blockNumber_ImageInfo,imageName_ImageInfo,time_stamps_ImageInfo,corrected_tstamps_ImageInfo,cFrame_Visual,displayStatus_Visual,worldTime_Visual,time_stamps_Visual,corrected_tstamps_Visual,ob_names,shift,latency,type,rotation,distance,block
0,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3730.998672,0.266204,3841.0,2.0,91.064651,3730.998659,0.266191,,True,,,,,
1,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3731.009716,0.277248,3842.0,2.0,91.075706,3731.009707,0.277238,,False,,,,,
2,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3731.020642,0.288174,3843.0,2.0,91.086632,3731.020630,0.288162,,False,,,,,
3,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3731.031608,0.299140,3844.0,2.0,91.097603,3731.031599,0.299130,,False,,,,,
4,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,0,grayCanvas,3731.042741,0.310273,3845.0,2.0,91.108734,3731.042731,0.310263,,False,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111175,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,1,img.1600x1000.date.2022-10-07_17-31-45.hitname...,5066.135132,1335.402663,114850.0,0.0,1426.197998,5066.135121,1335.402653,face_416,False,,,,,
111176,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,1,img.1600x1000.date.2022-10-07_17-31-45.hitname...,5066.146061,1335.413592,114851.0,0.0,1426.208862,5066.146050,1335.413581,face_416,False,,,,,
111177,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,1,img.1600x1000.date.2022-10-07_17-31-45.hitname...,5066.157166,1335.424698,114852.0,0.0,1426.219971,5066.157156,1335.424688,face_416,False,,,,,
111178,e0e948a0-f7fb-4f02-af14-e8e4c40a0e2b,1,img.1600x1000.date.2022-10-07_17-31-45.hitname...,5066.168250,1335.435782,114853.0,0.0,1426.231079,5066.168240,1335.435772,face_416,False,,,,,


In [234]:
len('2afa150a-914a-4afd-95f3-e7c065ffc4f1')

36

In [239]:
# path to save .csv with total images per block
total_img_file = os.path.join(t_path, 'total_unique_images_per_user.csv')
desc_file_streams = os.path.join(t_path, 'desc_files_streams.csv')

# progress bar format definitons
m_format = "{desc}:{bar}{percentage:3.0f}% {n_fmt}/{total_fmt} in {elapsed_s:.2f}s"
s_format = ("{desc}:{bar}{percentage:3.0f}% {n_fmt}/{total_fmt}{postfix} in {elapsed_s:.2f}s")
# main progress bar
main_bar = tqdm(
    os.listdir(r_path),
    desc="Processed",
    dynamic_ncols=True,
    mininterval=0.001,
    bar_format=m_format,
)
# for k in main_bar:
for file in main_bar:
    if file.lower().endswith('.xdf'):
        pbar = tqdm(
        range(6),
        mininterval=0.001,
        maxinterval=1,
        bar_format=s_format,)

        pbar.set_postfix(file=file)
        #### 1. Load the XDF file ####
        postfix = {"step": "1. Load the XDF file", "file": file}
        pbar.set_postfix(postfix)

        streams, _ = pyxdf.load_xdf(os.path.join(r_path, file))
        pbar.update(1)
        #### 2. Store selected stream info (only useful info)
        postfix = {"step": "2. Store selected streams info", "file": file}
        pbar.set_postfix(postfix)
        # store trigger-related stream data into df
        df = get_streams_data(streams, streams_keep=['ImageInfo','Visual'], time_ref_stream='openvibeSignal')
        # save eye_tracking related data
        df_eye = get_streams_data(streams, streams_keep=['HitObjectNames', 'HitPositionOnObjects','HeadTracking','EyeTrackingWorld','EyeTrackingLocal'])
        pbar.update(1)

        #### 3. Create triggers ####
        postfix = {"step": "3. Creating triggers from ImageInfo", "file": file}
        pbar.set_postfix(postfix)
        df_img, df_triggers, uid = create_triggers(df)
        pbar.update(1)

        #### 4. Save total number of images per block, per uid  ####
        postfix = {"step": "4 Saving total number of images per block, per uid", "file": file}
        pbar.set_postfix(postfix)
        img_unique = df_img[~df_img['imageName_ImageInfo'].str.contains('fixation|grayCan|Message')]
        total_images = img_unique.groupby('blockNumber_ImageInfo')['imageName_ImageInfo'].nunique().reset_index().rename(columns={"blockNumber_ImageInfo": "block_number", "imageName_ImageInfo": "total_img"})
        # convert column to numeric to drop the duplicates correctly
        total_images['block_number'] = pd.to_numeric(total_images['block_number'])

        # save total images for all participants
        if not os.path.isdir(t_path):
            os.mkdir(t_path)
        if not os.path.isdir(e_path):
            os.mkdir(e_path)
        desc_streams = describe_recordings(streams, file_name=file)

        #### 5. Create streams description per recording  ####
        postfix = {"step": "5. Creating triggers from ImageInfo", "file": file}
        pbar.set_postfix(postfix)
        if os.path.exists(desc_file_streams):
            df_desc_files = pd.read_csv(desc_file_streams)
            df_desc_files = pd.concat([df_desc_files,desc_streams], ignore_index=True)
            df_desc_files.reset_index(drop=True, inplace=True)
            df_desc_files.drop_duplicates(inplace=True, ignore_index=True)
        else:
            df_desc_files = desc_streams
        df_desc_files.to_csv(desc_file_streams, index=False)
        pbar.update(1)

        total_images['uid'] = uid # add colum to total images csv with uid
        if os.path.exists(total_img_file):
            df_total_unique_img = pd.read_csv(total_img_file)
            df_total_unique_img = pd.concat([df_total_unique_img,total_images], ignore_index=True)
            df_total_unique_img.reset_index(drop=True, inplace=True)
            df_total_unique_img.drop_duplicates(inplace=True, ignore_index=True)

        else:
            df_total_unique_img = total_images
        df_total_unique_img.to_csv(total_img_file, index=False)
        pbar.update(1)
        #### 6. Saving triggers and et files ####
        postfix = {"step": "6. Saving triggers and et files", "file": file}
        pbar.set_postfix(postfix)
        df_triggers.to_csv(os.path.join(t_path, 'trigger_file_' + uid +'.csv'), index=False)
        df_eye.to_csv(os.path.join(e_path, 'et_' + uid + '.csv'), index=False)
        pbar.update(1)
        pbar.set_postfix(file=file)
        pbar.close()

Processed:            0% 0/6 in 0.05s

:            0% 0/6 in 0.01s

:            0% 0/6 in 0.04s

:            0% 0/6 in 0.02s